In [1]:
import pickle
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore')

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [3]:
df = pd.read_csv('../data/tcga_processed.tsv.gz', sep='\t')

### some variants are repeated multiple times
df = df.drop(columns=['Cancer_type', 'Cancer_type_count', 'NMF_cluster'])

agg_columns = [
    'NMD_efficiency', 't_vaf', 't_depth', 'n_vaf', 'n_depth',
    'VAF_RNA', 'depth_RNA', 'VAF_DNA_RNA_ratio', 'tpm_unstranded_x'
]

# Group by all other columns except the ones to aggregate
group_by_columns = [col for col in df.columns if col not in agg_columns]

# Perform grouping and aggregation without dropping NaNs
df_unq = df.groupby(
    group_by_columns, dropna=False, as_index=False
).agg({col: 'mean' for col in agg_columns})

df_unq

,build,chromosome,start,end,Hugo_Symbol,Transcript_ID,HGVSc,HGVSp,PTC_pos_codon,PTC_to_start_codon,...,var_token_idx,NMD_efficiency,t_vaf,t_depth,n_vaf,n_depth,VAF_RNA,depth_RNA,VAF_DNA_RNA_ratio,tpm_unstranded_x
0,GRCh38,chr1,944753,944753,NOC2L,ENST00000327044,c.2191C>T,p.Gln731Ter,731,2193,...,2240,-0.508648,0.311111,45.0,0.0,43.0,0.442623,244.0,1.422717,76.8018
1,GRCh38,chr1,952113,952113,NOC2L,ENST00000327044,c.1218G>A,p.Trp406Ter,406,1218,...,1267,1.629509,0.390244,41.0,0.0,14.0,0.126126,111.0,0.323198,49.1731
2,GRCh38,chr1,1255304,1255304,UBE2J2,ENST00000349431,c.679G>T,p.Gly227Ter,227,681,...,898,0.047557,0.480000,50.0,0.0,32.0,0.464435,239.0,0.967573,47.4110
3,GRCh38,chr1,1338573,1338573,DVL1,ENST00000378888,c.1288G>T,p.Glu430Ter,430,1290,...,1572,1.505167,0.413043,46.0,0.0,63.0,0.145511,323.0,0.352289,50.0535
4,GRCh38,chr1,1387314,1387314,CCNL2,ENST00000400809,c.1480C>T,p.Arg494Ter,494,1482,...,1485,-0.424007,0.407692,130.0,0.0,152.0,0.546980,298.0,1.341649,27.1883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4088,GRCh38,chrX,152920717,152920717,ZNF185,ENST00000370268,c.622C>T,p.Gln208Ter,208,624,...,658,1.785660,0.328358,67.0,0.0,65.0,0.095238,42.0,0.290043,52.3474
4089,GRCh38,chrX,153650171,153650171,DUSP9,ENST00000342782,c.1021G>T,p.Glu341Ter,341,1023,...,1285,-1.360998,0.389313,131.0,0.0,185.0,1.000000,35.0,2.568627,16.9024
4090,GRCh38,chrX,154030948,154030948,MECP2,ENST00000303391,c.880C>T,p.Arg294Ter,294,882,...,1129,-1.166585,0.406977,86.0,0.0,70.0,0.913580,81.0,2.244797,11.5544
4091,GRCh38,chrX,154354015,154354015,FLNA,ENST00000369850,c.5586C>A,p.Tyr1862Ter,1862,5586,...,5834,0.332048,0.904762,189.0,0.0,134.0,0.718750,32.0,0.794408,63.2541


### Splits

In [4]:
test_var_ids = df_unq[df_unq['chromosome'].isin(['chr20', 'chr21', 'chr22'])]['var_id'].values.tolist()
test_var_ids = list(set(test_var_ids))
test_var_ids[0:5], len(test_var_ids)

(['ENST00000330029:c.112C>T',
  'ENST00000263121:c.964C>T',
  'ENST00000372720:c.733G>T',
  'ENST00000248958:c.487C>T',
  'ENST00000371711:c.1159G>T'],
 232)

In [5]:
val_var_ids = df_unq[df_unq['chromosome']=='chr19']['var_id'].values.tolist()
val_var_ids = list(set(val_var_ids))
val_var_ids[0:5], len(val_var_ids)

(['ENST00000320936:c.130C>T',
  'ENST00000294618:c.4402C>T',
  'ENST00000263097:c.906C>G',
  'ENST00000160298:c.3418G>T',
  'ENST00000040663:c.898C>T'],
 210)

In [6]:
train_var_ids = list(set(df_unq.var_id.values.tolist()) - set(val_var_ids) - set(test_var_ids))
train_var_ids[0:5], len(train_var_ids)

(['ENST00000344936:c.2215G>T',
  'ENST00000448903:c.2797G>T',
  'ENST00000314340:c.346G>T',
  'ENST00000324545:c.2224C>T',
  'ENST00000296318:c.1576C>T'],
 3651)

In [7]:
# create dictionaries with NMD_efficiency as values
test_dict = {}
val_dict = {}
train_dict = {}
var_pos_idx_dict = {}

for index, row in df_unq.iterrows():
    if row['var_id'] in test_var_ids:
        test_dict[row['var_id']] = row['NMD_efficiency']
    elif row['var_id'] in val_var_ids:
        val_dict[row['var_id']] = row['NMD_efficiency']
    else:
        train_dict[row['var_id']] = row['NMD_efficiency']

    var_pos_idx_dict[row['var_id']] = row['var_token_idx']

### Read embeddings

In [8]:
with open('../data/tcga_ref_embeds.pkl', 'rb') as file:
    ref_embeds = pickle.load(file)

with open('../data/tcga_alt_embeds.pkl', 'rb') as file:
    alt_embeds = pickle.load(file)

# create alt - ref embeds
altref_embeds = {}

for var_id, alt_embed in alt_embeds.items():
    altref_embeds[var_id] = alt_embed - ref_embeds[var_id]

### Train and evaluate

In [9]:
import logging
import sys
from collections import OrderedDict
from typing import Any, Dict, List, Optional

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr, pearsonr
import torch.optim as optim
import pandas as pd

# ---------------------------
# Hyperparameters
# ---------------------------
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 5e-4
HIDDEN_DIMS_phi = [8, 8]
HIDDEN_DIMS_rho = [4, 4]
DROPOUT = 0.25
N_EPOCHS = 50
EARLY_STOPPING_PATIENCE = 10
VALIDATION_INTERVAL = 5  # Perform validation every 5 epochs

# ---------------------------
# Dataset Definition
# ---------------------------
class DeepSetDataset(Dataset):
    def __init__(self, embeds_dict, target_dict, var_pos_idx_dict=None):
        self.ids = list(target_dict.keys())
        self.embeds = [embeds_dict[i] for i in self.ids]
        self.targets = [target_dict[i] for i in self.ids]
        self.var_pos_idx = [var_pos_idx_dict[i] if var_pos_idx_dict else None for i in self.ids]

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        embed = self.embeds[idx]
        target = torch.tensor(self.targets[idx], dtype=torch.float32)
        var_pos_idx = self.var_pos_idx[idx]
        return embed, target, var_pos_idx


def collate_fn(batch):
    embeds, targets, var_pos_idx = zip(*batch)
    embeds = [torch.as_tensor(embed, dtype=torch.float32) for embed in embeds]
    embeds_padded = torch.nn.utils.rnn.pad_sequence(embeds, batch_first=True, padding_value=0.0)
    targets = torch.tensor(targets, dtype=torch.float32)
    mask = (embeds_padded != 0).any(dim=-1).float()
    return embeds_padded, targets, mask, var_pos_idx

# ---------------------------
# DeepSet Model with Masking
# ---------------------------
class DeepSetModel(nn.Module):
    def __init__(
        self,
        input_dim: int,
        phi_hidden_dims: List[int],
        rho_hidden_dims: List[int],
        output_dim: int = 1,
        activation: str = "ReLU",
        pool: str = "mean",
        dropout: Optional[float] = None,
    ):
        super().__init__()
        self.activation = getattr(nn, activation)()
        self.pool = pool
        self.dropout = nn.Dropout(dropout) if dropout else None
        
        # Phi network
        phi = []
        in_dim = input_dim
        for i, dim in enumerate(phi_hidden_dims):
            phi.append((f"phi_linear_{i}", nn.Linear(in_dim, dim)))
            if dropout:
                phi.append((f"phi_dropout_{i}", self.dropout))
            phi.append((f"phi_activation_{i}", self.activation))
            in_dim = dim
        self.phi = nn.Sequential(OrderedDict(phi))
        
        # Rho network
        rho = []
        for i, dim in enumerate(rho_hidden_dims[:-1]):
            rho.append((f"rho_linear_{i}", nn.Linear(in_dim, dim)))
            if dropout:
                rho.append((f"rho_dropout_{i}", self.dropout))
            rho.append((f"rho_activation_{i}", self.activation))
            in_dim = dim
        rho.append((f"rho_linear_{len(rho_hidden_dims) - 1}", nn.Linear(in_dim, output_dim)))
        self.rho = nn.Sequential(OrderedDict(rho))

    def forward(self, x, mask, var_pos_idx=None):
        x = self.phi(x)
        
        if self.pool == "sum":
            x = (x * mask.unsqueeze(-1)).sum(dim=1)
        elif self.pool == "max":
            x = (x * mask.unsqueeze(-1)).masked_fill(mask.unsqueeze(-1) == 0, float('-inf')).max(dim=1).values
        elif self.pool == "mean":
            x = (x * mask.unsqueeze(-1)).sum(dim=1) / (mask.sum(dim=1, keepdim=True) + 1e-8)
        elif self.pool == "token" and var_pos_idx is not None:
            x = torch.stack([x[i, idx] for i, idx in enumerate(var_pos_idx)])
        else:
            raise ValueError(f"Unsupported pooling operation: {self.pool}")
        
        return self.rho(x)


# ---------------------------
# Evaluation Function
# ---------------------------
def evaluate_regression_metrics(y_true, y_pred):
    y_true = y_true.cpu().numpy().flatten()
    y_pred = y_pred.cpu().numpy().flatten()
    loss = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(loss)
    r2 = r2_score(y_true, y_pred)
    
    try:
        spearman_corr, _ = spearmanr(y_true, y_pred)
    except ValueError:
        spearman_corr = np.nan

    try:
        pearson_corr, _ = pearsonr(y_true, y_pred)
    except ValueError:
        pearson_corr = np.nan
    
    return {
        'loss': loss,
        'spearman_corr': spearman_corr,
        'pearson_corr': pearson_corr,
        'mae': mae,
        'rmse': rmse,
        'r2': r2
    }


# ---------------------------
# Training Function
# ---------------------------
def train_model(train_dict, val_dict, test_dict, embeds_dict, var_pos_idx_dict=None, pool='mean'):
    train_loader = DataLoader(DeepSetDataset(embeds_dict, train_dict, var_pos_idx_dict), batch_size=32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(DeepSetDataset(embeds_dict, val_dict, var_pos_idx_dict), batch_size=32, collate_fn=collate_fn)
    test_loader = DataLoader(DeepSetDataset(embeds_dict, test_dict, var_pos_idx_dict), batch_size=32, collate_fn=collate_fn)
    
    model = DeepSetModel(
        input_dim=embeds_dict[list(embeds_dict.keys())[0]].shape[1],
        phi_hidden_dims=HIDDEN_DIMS_phi,
        rho_hidden_dims=HIDDEN_DIMS_rho,
        pool=pool,
        dropout=DROPOUT
    )
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = nn.MSELoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    metrics = {'epoch': [], 'phase': [], 'loss': [], 'spearman_corr': [], 'pearson_corr':[], 'mae': [], 'rmse': [], 'r2': []}
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(1, N_EPOCHS + 1):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                loader = train_loader
            else:
                model.eval()
                loader = val_loader
            
            all_preds, all_targets = [], []
            running_loss = 0.0
            
            for embeds, targets, mask, var_pos_idx in loader:
                embeds, targets, mask = embeds.to(device), targets.to(device), mask.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(embeds, mask, var_pos_idx).squeeze()
                    loss = criterion(outputs, targets)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * embeds.size(0)
                all_preds.append(outputs.detach())
                all_targets.append(targets.detach())
            
            epoch_loss = running_loss / len(loader.dataset)
            all_preds = torch.cat(all_preds)
            all_targets = torch.cat(all_targets)
            
            epoch_metrics = evaluate_regression_metrics(all_targets, all_preds)
            epoch_metrics['loss'] = epoch_loss
            
            # Early stopping for validation phase
            if phase == 'val':
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= EARLY_STOPPING_PATIENCE:
                        print(f"Early stopping at epoch {epoch}")
                        metrics_df = pd.DataFrame(metrics)
                        test_metrics = evaluate_regression_metrics(all_targets, all_preds)
                        test_metrics['loss'] = epoch_loss
                        return metrics_df, test_metrics
            
            # Log metrics every 5 epochs
            if epoch % 5 == 0:
                print(
                    f"[Epoch {epoch:03d}] Phase: {phase:5s} | "
                    f"Loss: {epoch_metrics['loss']:.4f} | "
                    f"Spearman: {epoch_metrics['spearman_corr']:.4f} | "
                    f"Pearson: {epoch_metrics['pearson_corr']:.4f} | "
                    f"MAE: {epoch_metrics['mae']:.4f} | "
                    f"RMSE: {epoch_metrics['rmse']:.4f} | "
                    f"R²: {epoch_metrics['r2']:.4f}"
                )
                metrics['epoch'].append(epoch)
                metrics['phase'].append(phase)
                for key, value in epoch_metrics.items():
                    metrics[key].append(value)
    
    # Final Test Evaluation
    model.eval()
    all_preds, all_targets = [], []
    running_loss = 0.0
    
    for embeds, targets, mask, var_pos_idx in test_loader:
        embeds, targets, mask = embeds.to(device), targets.to(device), mask.to(device)
        outputs = model(embeds, mask, var_pos_idx).squeeze()
        loss = criterion(outputs, targets)
        
        running_loss += loss.item() * embeds.size(0)
        all_preds.append(outputs.detach())
        all_targets.append(targets.detach())
    
    epoch_loss = running_loss / len(test_loader.dataset)
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    
    test_metrics = evaluate_regression_metrics(all_targets, all_preds)
    test_metrics['loss'] = epoch_loss
    
    metrics_df = pd.DataFrame(metrics)
    return metrics_df, test_metrics


In [10]:
### Ref embeds

print("START TRAINING REF MEAN")
ref_mean_metrics_df, ref_mean_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=ref_embeds,
        pool='mean'  
    )
ref_mean_metrics_df.to_csv('../res/metrics/per_epoch_deepset/ref_mean_history_df.csv')

print("START TRAINING REF MAX")
ref_max_metrics_df, ref_max_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=ref_embeds,
        pool='max'  
    )
ref_max_metrics_df.to_csv('../res/metrics/per_epoch_deepset/ref_max_metrics_df.csv')

print("START TRAINING REF TOKEN")
ref_token_metrics_df, ref_token_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=ref_embeds,
        pool='token',
        var_pos_idx_dict=var_pos_idx_dict
    )
ref_token_metrics_df.to_csv('../res/metrics/per_epoch_deepset/ref_token_metrics_df.csv')

print("START TRAINING REF SUM")
ref_sum_metrics_df, ref_sum_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=ref_embeds,
        pool='sum'  
    )
ref_sum_metrics_df.to_csv('../res/metrics/per_epoch_deepset/ref_sum_metrics_df.csv')

START TRAINING REF MEAN
[Epoch 005] Phase: train | Loss: 1.5912 | Spearman: 0.0930 | Pearson: 0.0777 | MAE: 1.0420 | RMSE: 1.2614 | R²: -0.0055
[Epoch 005] Phase: val   | Loss: 1.7104 | Spearman: 0.2835 | Pearson: 0.3280 | MAE: 1.0405 | RMSE: 1.3078 | R²: 0.0251
[Epoch 010] Phase: train | Loss: 1.4423 | Spearman: 0.2991 | Pearson: 0.3015 | MAE: 0.9877 | RMSE: 1.2010 | R²: 0.0886
[Epoch 010] Phase: val   | Loss: 1.5250 | Spearman: 0.4085 | Pearson: 0.4143 | MAE: 1.0059 | RMSE: 1.2349 | R²: 0.1308
[Epoch 015] Phase: train | Loss: 1.4263 | Spearman: 0.3190 | Pearson: 0.3147 | MAE: 0.9816 | RMSE: 1.1943 | R²: 0.0988
[Epoch 015] Phase: val   | Loss: 1.4949 | Spearman: 0.4306 | Pearson: 0.4316 | MAE: 1.0048 | RMSE: 1.2227 | R²: 0.1479
[Epoch 020] Phase: train | Loss: 1.4333 | Spearman: 0.3255 | Pearson: 0.3081 | MAE: 0.9812 | RMSE: 1.1972 | R²: 0.0944
[Epoch 020] Phase: val   | Loss: 1.4692 | Spearman: 0.4449 | Pearson: 0.4445 | MAE: 0.9927 | RMSE: 1.2121 | R²: 0.1626
[Epoch 025] Phase: trai

In [11]:
### Alt embeds

print("START TRAINING ALT MEAN")
alt_mean_metrics_df, alt_mean_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=alt_embeds,
        pool='mean'  
    )
alt_mean_metrics_df.to_csv('../res/metrics/per_epoch_deepset/alt_mean_metrics_df.csv')

print("START TRAINING ALT MAX")
alt_max_metrics_df, alt_max_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=alt_embeds,
        pool='max'  
    )
alt_max_metrics_df.to_csv('../res/metrics/per_epoch_deepset/alt_max_metrics_df.csv')

print("START TRAINING ALT TOKEN")
alt_token_metrics_df, alt_token_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=alt_embeds,
        pool='token',
        var_pos_idx_dict=var_pos_idx_dict
    )
alt_token_metrics_df.to_csv('../res/metrics/per_epoch_deepset/alt_token_metrics_df.csv')

print("START TRAINING ALT SUM")
alt_sum_metrics_df, alt_sum_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=alt_embeds,
        pool='sum'  
    )
alt_sum_metrics_df.to_csv('../res/metrics/per_epoch_deepset/alt_sum_metrics_df.csv')

START TRAINING ALT MEAN
[Epoch 005] Phase: train | Loss: 1.6969 | Spearman: 0.0598 | Pearson: 0.0301 | MAE: 1.0633 | RMSE: 1.3027 | R²: -0.0722
[Epoch 005] Phase: val   | Loss: 1.7039 | Spearman: 0.3763 | Pearson: 0.3787 | MAE: 1.0497 | RMSE: 1.3053 | R²: 0.0288
[Epoch 010] Phase: train | Loss: 1.5557 | Spearman: 0.1841 | Pearson: 0.1876 | MAE: 1.0194 | RMSE: 1.2473 | R²: 0.0170
[Epoch 010] Phase: val   | Loss: 1.5447 | Spearman: 0.4373 | Pearson: 0.4366 | MAE: 0.9931 | RMSE: 1.2429 | R²: 0.1195
[Epoch 015] Phase: train | Loss: 1.4891 | Spearman: 0.2725 | Pearson: 0.2714 | MAE: 0.9895 | RMSE: 1.2203 | R²: 0.0591
[Epoch 015] Phase: val   | Loss: 1.4485 | Spearman: 0.4581 | Pearson: 0.4572 | MAE: 0.9486 | RMSE: 1.2035 | R²: 0.1744
[Epoch 020] Phase: train | Loss: 1.4246 | Spearman: 0.3320 | Pearson: 0.3294 | MAE: 0.9621 | RMSE: 1.1935 | R²: 0.0999
[Epoch 020] Phase: val   | Loss: 1.4038 | Spearman: 0.4647 | Pearson: 0.4683 | MAE: 0.9319 | RMSE: 1.1848 | R²: 0.1998
[Epoch 025] Phase: trai

In [12]:
### Alt - Ref embeds

print("START TRAINING ALTREF MEAN")
altref_mean_metrics_df, altref_mean_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=altref_embeds,
        pool='mean'  
    )
altref_mean_metrics_df.to_csv('../res/metrics/per_epoch_deepset/altref_mean_metrics_df.csv')

print("START TRAINING ALTREF MAX")
altref_max_metrics_df, altref_max_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=altref_embeds,
        pool='max'  
    )
altref_max_metrics_df.to_csv('../res/metrics/per_epoch_deepset/altref_max_metrics_df.csv')

print("START TRAINING ALTREF TOKEN")
altref_token_metrics_df, altref_token_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=altref_embeds,
        pool='token',
        var_pos_idx_dict=var_pos_idx_dict
    )
altref_token_metrics_df.to_csv('../res/metrics/per_epoch_deepset/altref_token_metrics_df.csv')

print("START TRAINING ALTREF SUM")
altref_sum_metrics_df, altref_sum_test_metrics = train_model(
        train_dict=train_dict,
        val_dict=val_dict,
        test_dict=test_dict,
        embeds_dict=altref_embeds,
        pool='sum'  
    )
altref_sum_metrics_df.to_csv('../res/metrics/per_epoch_deepset/altref_sum_metrics_df.csv')

START TRAINING ALTREF MEAN
[Epoch 005] Phase: train | Loss: 1.5918 | Spearman: 0.0728 | Pearson: 0.0310 | MAE: 1.0506 | RMSE: 1.2616 | R²: -0.0058
[Epoch 005] Phase: val   | Loss: 1.7403 | Spearman: 0.3977 | Pearson: 0.3455 | MAE: 1.0793 | RMSE: 1.3192 | R²: 0.0080
[Epoch 010] Phase: train | Loss: 1.5116 | Spearman: 0.2476 | Pearson: 0.2381 | MAE: 1.0206 | RMSE: 1.2295 | R²: 0.0449
[Epoch 010] Phase: val   | Loss: 1.6710 | Spearman: 0.3605 | Pearson: 0.3105 | MAE: 1.0494 | RMSE: 1.2927 | R²: 0.0475
[Epoch 015] Phase: train | Loss: 1.3916 | Spearman: 0.3841 | Pearson: 0.3670 | MAE: 0.9686 | RMSE: 1.1797 | R²: 0.1207
[Epoch 015] Phase: val   | Loss: 1.5631 | Spearman: 0.4143 | Pearson: 0.3714 | MAE: 1.0065 | RMSE: 1.2502 | R²: 0.1091
[Epoch 020] Phase: train | Loss: 1.3376 | Spearman: 0.4373 | Pearson: 0.4014 | MAE: 0.9425 | RMSE: 1.1565 | R²: 0.1548
[Epoch 020] Phase: val   | Loss: 1.5185 | Spearman: 0.4235 | Pearson: 0.3837 | MAE: 0.9751 | RMSE: 1.2323 | R²: 0.1345
[Epoch 025] Phase: t

In [13]:
mean_metrics = pd.DataFrame({ 'ref':ref_mean_test_metrics,
                'alt': alt_mean_test_metrics, 
                'altref': altref_mean_test_metrics})

mean_metrics.to_csv('../res/metrics/test_deepset/mean_metrics.csv')
mean_metrics

,ref,alt,altref
loss,1.452732,1.337618,1.377145
spearman_corr,0.450857,0.474312,0.479753
pearson_corr,0.447302,0.445301,0.401554
mae,0.982753,0.888658,0.916866
rmse,1.205294,1.156554,1.173518
r2,0.171968,0.177735,0.153436


In [14]:
max_metrics = pd.DataFrame({ 'ref':ref_max_test_metrics,
                'alt': alt_max_test_metrics, 
                'altref': altref_max_test_metrics})

max_metrics.to_csv('../res/metrics/test_deepset/max_metrics.csv')
max_metrics

,ref,alt,altref
loss,1.509301,1.535648,1.672286
spearman_corr,0.448156,0.418089,0.191863
pearson_corr,0.458341,0.429344,0.139085
mae,0.981189,0.996700,1.007356
rmse,1.228536,1.239213,1.293169
r2,0.139725,0.124707,-0.027994


In [15]:
token_metrics = pd.DataFrame({'ref':ref_token_test_metrics,
                'alt': alt_token_test_metrics, 
                'altref': altref_token_test_metrics})

token_metrics.to_csv('../res/metrics/test_deepset/token_metrics.csv')
token_metrics

,ref,alt,altref
loss,1.699620,1.726859,1.709100
spearman_corr,0.198805,0.192727,0.213693
pearson_corr,0.181129,0.133779,0.171494
mae,1.067796,1.056331,1.048893
rmse,1.303695,1.314100,1.307325
r2,0.031246,0.015721,0.025843


In [16]:
sum_metrics = pd.DataFrame({'ref':ref_sum_test_metrics,
                'alt': alt_sum_test_metrics, 
                'altref': altref_sum_test_metrics})

sum_metrics.to_csv('../res/metrics/test_deepset/sum_metrics.csv')
sum_metrics

,ref,alt,altref
loss,2.766022,2.096702,1.687239
spearman_corr,0.171579,NaN,-0.005422
pearson_corr,0.124623,NaN,-0.053877
mae,1.258030,1.096062,1.044315
rmse,1.663136,1.447999,1.298938
r2,-0.700339,-0.288892,-0.037185
